In [2]:
import sys
import os
sys.path.append(os.path.abspath('./../../../..'))
from cell_extractor.CellDetectorBase import CellDetectorBase
from pipeline.lib.annotation_layer import Cell,random_string
import numpy as np
from random import sample
from pipeline.lib.UrlGenerator import UrlGenerator
from cell_extractor.BorderFinder import BorderFinder
from cell_extractor.CellDetector import MultiThresholdDetector
def numpy_to_json(data,color_hex = None,category = 'Round3_Sure'):
    cells = []
    for coord in data:
        cell = Cell(np.array(coord,dtype=float),random_string())
        cell.category = category
        cell.coord[-1]=cell.coord[-1]+0.5
        if hasattr(cell,'description'):
            del cell.description
        cell_json = cell.to_json()
        if color_hex is not None:
            cell_json["props"] = [color_hex]
        cells.append(cell_json)
    return cells

def create_QC_url(animal,sures,unsures,title):
    urlGen = UrlGenerator()
    image_layer = 'precomputed://https://activebrainatlas.ucsd.edu/data/'+animal+'/neuroglancer_data/'
    urlGen.add_stack_image(animal,channel=1)
    urlGen.add_stack_image(animal,channel=2,color='red')
    urlGen.add_stack_image(animal,channel=3,color='green')
    urlGen.add_annotation_layer('Sure',annotations = sures)
    urlGen.add_annotation_layer('Unsure',annotations = unsures)
    return urlGen.add_to_database(title,34)

def get_multi_threshold_sure_and_unsure(animal):
    detector = MultiThresholdDetector(animal,round = 2)
    sure = detector.get_sures()
    unsure = detector.get_unsures()
    return sure,unsure

def get_single_threshold_sure_and_unsure(animal,threshold):
    detector = CellDetectorBase(animal,round = 2,segmentation_threshold=threshold)
    detections = detector.load_detections()
    sures = detections[detections.predictions==2]
    unsure = detections[detections.predictions==0]
    return sures,unsure

def print_multi_threshold_sure_and_unsure_count(animal):
    sure,unsure = get_multi_threshold_sure_and_unsure(animal)
    print(len(sure),len(unsure))
    
def print_single_threshold_sure_and_unsure_count(animal,threshold = 2000):
    sure,unsure = get_single_threshold_sure_and_unsure(animal,threshold)
    print(len(sure),len(unsure))

def generate_QC_link(animal,get_sure_and_unsure,sample = None):
    finder = BorderFinder(animal)
    sure,unsure = get_sure_and_unsure()
    detector = CellDetectorBase(animal,round = 2)
    detections = detector.load_detections()
    sure = detections[detections.predictions==2]
    unsure = detections[detections.predictions==0]
    sure.rename(columns = {'x' : 'col', 'y' : 'row'}, inplace = True)
    unsure.rename(columns = {'x' : 'col', 'y' : 'row'}, inplace = True)
    _,sure = finder.find_border_cells(sure)
    _,unsure = finder.find_border_cells(unsure)
    if sample is not None:
        sure_data_sample = sure[['col','row','section']].sample(sample).sort_values('section').to_numpy().tolist()
    else:
        sure_data_sample = sure[['col','row','section']].sort_values('section').to_numpy().tolist()
    sure_data_sample = sure[['col','row','section']].sort_values('section').to_numpy().tolist()
    sure_data = sure[['col','row','section']].sort_values('section').to_numpy().tolist()
    unsure_data = unsure[['col','row','section']].sort_values('section').to_numpy().tolist()
    sure_cells_sample = numpy_to_json(sure_data_sample,category = 'Round3_Sure')
    sure_cells = numpy_to_json(sure_data,category = 'Round3_Sure')
    unsure_cells = numpy_to_json(unsure_data,color_hex='#1d66db',category = 'Round3_Unsure')

    sc_id = create_QC_url(animal,sure_cells_sample,unsure_cells,'Julian QC3 '+animal)
    false_negative_id = create_QC_url(animal,sure_cells,unsure_cells,'Julian false negative '+animal)
    print(f'Julian QC url id {animal} : {sc_id}')
    print(f'Julian false negative url id {animal} : {false_negative_id}')

    sc_id = create_QC_url(animal,sure_cells_sample,unsure_cells,'Marissa QC3 '+animal)
    false_negative_id = create_QC_url(animal,sure_cells,unsure_cells,'Marissa false negative '+animal)
    print(f'Marissa QC url id {animal} : {sc_id}')
    print(f'Marissa false negative url id {animal} : {false_negative_id}')

In [4]:
animal = 'DK39'
finder = BorderFinder(animal)
# detector = MultiThresholdDetector(animal,round = 2)
# sure = detector.get_sures()
# unsure = detector.get_unsures()
detector = CellDetectorBase(animal,round = 2)
detections = detector.load_detections()
sure = detections[detections.predictions==2]
unsure = detections[detections.predictions==0]
sure.rename(columns = {'x' : 'col', 'y' : 'row'}, inplace = True)
unsure.rename(columns = {'x' : 'col', 'y' : 'row'}, inplace = True)
border_cell,sure = finder.find_border_cells(sure)
border_cell,unsure = finder.find_border_cells(unsure)
# sure_data_sample = sure[['col','row','section']].sample(500).sort_values('section').to_numpy().tolist()
sure_data_sample = sure[['col','row','section']].sort_values('section').to_numpy().tolist()
sure_data = sure[['col','row','section']].sort_values('section').to_numpy().tolist()
unsure_data = unsure[['col','row','section']].sort_values('section').to_numpy().tolist()
sure_cells_sample = numpy_to_json(sure_data_sample,category = 'Round3_Sure')
sure_cells = numpy_to_json(sure_data,category = 'Round3_Sure')
unsure_cells = numpy_to_json(unsure_data,color_hex='#1d66db',category = 'Round3_Unsure')

sc_id = create_QC_url(animal,sure_cells_sample,unsure_cells,'Julian QC3 '+animal)
false_negative_id = create_QC_url(animal,sure_cells,unsure_cells,'Julian false negative '+animal)
print(f'Julian QC url id {animal} : {sc_id}')
print(f'Julian false negative url id {animal} : {false_negative_id}')

# sc_id = create_QC_url(animal,sure_cells_sample,unsure_cells,'Marissa QC3 '+animal)
# false_negative_id = create_QC_url(animal,sure_cells,unsure_cells,'Marissa false negative '+animal)
# print(f'Marissa QC url id {animal} : {sc_id}')
# print(f'Marissa false negative url id {animal} : {false_negative_id}')

/usr/local/share/pipeline/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Julian QC url id DK39 : 640
Julian false negative url id DK39 : 641


In [ ]:
print_single_threshold_sure_and_unsure_count('DK41')